In [1]:
import pandas as pd 
import numpy as np
import os 
os.environ["WANDB_DISABLED"] = "true"

In [8]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 20.3 MB/s eta 0:00:00


In [2]:
SAVE_TXT = True

In [3]:
df_tr = pd.read_csv("data/movies_train.csv")
df_test = pd.read_csv("data/movies_test.csv")

In [4]:
if SAVE_TXT:
    df_tr["concat"] = df_tr.apply(lambda x: f"Title:{x.title} Text:{x.text}", axis=1)
    df_test["concat"] = df_test.apply(lambda x: f"Title:{x.title} Text:{x.text}", axis=1)

    ids = np.random.choice(list(range(len(df_tr))), size=500, replace=False)
    ids.sort()

    ids_tr = list(set(list(range(len(df_tr)))) - set(ids))

    text_tr = df_tr.iloc[ids_tr].concat.values.tolist() + df_test.concat.values.tolist()
    text_dev = df_tr.iloc[ids].concat.values.tolist()

    print(len(text_tr), len(text_dev))


    with open("data/movies_text_tr.txt", "w") as f:
        for t in text_tr:
            f.write(t + '\n')

    with open("data/movies_text_dev.txt", "w") as f:
        for t in text_dev:
            f.write(t + '\n')

89500 500


# Roberta

In [4]:
!python run_mlm.py \
    --model_name_or_path roberta-large \
    --train_file data/movies_text_tr.txt \
    --validation_file data/movies_text_dev.txt \
    --per_device_train_batch_size 32 \
    --per_device_eval_batch_size 16 \
    --do_train \
    --do_eval \
    --fp16 True \
    --max_seq_len 128 \
    --num_train_epochs 30\
    --line_by_line True \
    --pad_to_max_length False \
    --evaluation_strategy 'steps' \
    --metric_for_best_model 'eval_accuracy' \
    --save_total_limit 3 \
    --logging_steps 500 \
    --eval_steps 500 \
    --load_best_model_at_end True \
    --output_dir 'roberta-large-movies'\
    --overwrite_output_dir True \
    
# 

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
07/17/2023 14:57:23 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
07/17/2023 14:57:23 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=500,
evaluation_strategy=steps,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_par